## Agile Hardware Design
***
# FP Conclusion + Pattern Matching

<img src="../resource/logo.svg" alt="agile hardware design logo" style="float:right"/>

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 228A](https://classes.soe.ucsc.edu/cse228a/Spring25/)

## Plan for Today

* FP conclusion: flatMap, filter, sum
* Pattern matching
* Gracefully handling Option

## Loading The Chisel Library Into a Notebook

In [ ]:
interp.configureCompiler(_.settings.processArguments(List("-Wconf:cat=deprecation:s"), true))
interp.load.module(os.Path(s"${System.getProperty("user.dir")}/../resource/chisel_deps.sc"))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

## Scala `flatMap`

* Like `map`, but concatenates output of function's
  * More direct than calling `map` and then `flatten`
* Helpful for aggregating results of map when they are a collection
* Also useful if map will return 0 or more elements

In [ ]:
val l = 0 until 5
// Seq.fill(2)(3)
l map { i => Seq.fill(i)(i) }
(l map { i: Int => Seq.fill(i)(i) }).flatten
l flatMap { i => Seq.fill(i)(i) }

// l flatMap { i => if (i%2 == 0) Seq(i) else Seq()}

## Visualizing `map` on Collections

<img src="images/mapOnC.svg" alt="map on collections" style="width:70%;margin-left:auto;margin-right:auto"/>

## Visualizing `flatMap`

<img src="images/flatmap.svg" alt="flatmap" style="width:70%;margin-left:auto;margin-right:auto"/>

## Applying Predicates in Scala

* A _predicate_ is a function that given a single element, returns a `Boolean`
* `filter` - elements persists to output collection only if predicate returns true
* `forall` - true if and only if predicate is true for all elements
* `exists` - true if predicate is true for at least one element

In [ ]:
def isEven(x: Int): Boolean = x % 2 == 0
val l = 0 until 5

l filter isEven
// l filter { x => !isEven(x) }
l filterNot isEven
l forall isEven
l exists isEven

## Visualizing `filter`

<img src="images/filter.svg" alt="filter viz" style="width:40%;margin-left:auto;margin-right:auto"/>

## Example: Prime Seive in Scala

In [ ]:
def multipleOf(a: Int)(b: Int): Boolean = (b % a == 0)

def removeMultiplesOfX(l: Seq[Int], x: Int) = l filterNot multipleOf(x)

val allNums = 2 until 100

// println(allNums filterNot multipleOf(5))
// println(removeMultiplesOfX(allNums, 5))

def seive(s: Seq[Int]): Seq[Int] = {
    if (s.isEmpty) Seq()
    else Seq(s.head) ++ seive(removeMultiplesOfX(s.tail, s.head))
}

println(seive(allNums))

## Scala Has Common Reductions Built-in

* `sum`, `product`, `min`, `max`

In [ ]:
val l = 0 until 5
l reduce { _ + _ }
l.sum
l reduce { _ * _ }
l.product
l reduce { _ min _ }
l.min
l reduce { _ max _ }
l.max

## Aside: Learn To Things the Scala Way

* In many cases, Scala provides methods for things other languages wouldn't, such as:
    * `isEmpty`/`nonEmpty`, `to`/`until`, `filter`/`filterNot`, `foldLeft`/`foldRight`
* _Problem:_ Language newcomers may not know about all of the features or common idioms
* _Solution A:_ Yet another reason to use an IDE, as it may recognize common launguage misuses
* _Solution B:_ Code reviews and looking at the code of others can help
* _Solution C:_ Read [Scala Collections Tips and Tricks](https://pavelfatin.com/scala-collections-tips-and-tricks/) by Pavel Fatin
    * Linked from course website under _Reference $\rightarrow$ Scala_

## Example: Using FP to do Matrix Multiplication (1/2)

In [ ]:
// matrix in row-major layout
val mat = Seq.tabulate(4,4){ (i,j) => i+j }

def grabCol(m: Seq[Seq[Int]], i: Int) = m map { row => row(i) }
grabCol(mat,1)

def dotP(a: Seq[Int], b: Seq[Int]) = a.zip(b).map{ case (a_i,b_i) => a_i * b_i}.sum

def matMul(a: Seq[Seq[Int]], b: Seq[Seq[Int]]) = a map {
    rowOfA => (0 until rowOfA.size) map { colIndex => dotP(rowOfA, grabCol(b, colIndex)) }
}

matMul(mat, mat)

## Example: Using FP to do Matrix Multiplication (2/2)

In [ ]:
def matMul(a: Seq[Seq[Int]], b: Seq[Seq[Int]]) = {
    (0 until a.size) map { i => {
        (0 until b.head.size) map {j => 
            dotP(a(i), grabCol(b, j))
        }
    }}
}

// def matMul(a: Seq[Seq[Int]], b: Seq[Seq[Int]]) = Seq.tabulate(a.length, b.head.length){
//     (i,j) => dotP(a(i), grabCol(b,j))
// }

matMul(mat, mat)

## Pattern Matching in Scala

* Can gracefully scale from replacing simple `if/else` and `switch` cases to more sophisticated searches
* Start block with `match` and list matches with `case`
* Can use `|` for or
* Can use `if` to specify condition
* Can use `_` for default (matched nothing above)

In [ ]:
val x = 0

x match {
    case 0 => "0"
    case 1 | 3 => "nah"
    case y if (y%2 == 0) => "even"
    case 5 => "found it!"
    case _ => "other"
}

## Can Match on Case Classes

* Can match on type as a whole, or even set fields

In [ ]:
abstract class Vehicle

case class helicopter(color: String, driver: String) extends Vehicle

case class submarine(color: String, driver: String) extends Vehicle

val movers = Seq(helicopter("grey", "Marta"), helicopter("blue", "Laura"), submarine("yellow", "Paul"))

movers foreach {v => v match {
    case h: helicopter => println(s"${h.color} helicopter")
    case s: submarine => println(s"${s.color} submarine")
}}

movers foreach { _ match {
    case helicopter("blue", driver) => println(s"$driver has a blue helicopter")
    case s: submarine if (s.color != "yellow") => println(s"${s.driver}'s ${s.color} submarine")
    case _ => println("didn't match")
}}

## More Graceful Interactions with `Option`

* Many Scala operations pass over None

In [ ]:
val l = Seq.tabulate(5)(i => if (i % 2 == 1) Some(i) else None)

l foreach { x =>
    if (x.isDefined) println(x.get)
}

// l(1).getOrElse(-1)

// l.flatten

// l foreach { _ match {
//     case Some(i) => println(i)
//     case None => println("was empty")
// }}

// l(0) foreach println

## Project Overview

### GOAL: gain experience developing/revising a generator

### Main Details
* Working in pairs (or individually)
* Pick an idea to build a generator for (contact instructor if need suggestion)
* Will _propose/design/develop/test/optimize/revise/document/present_ generator
* Ballpark for size/complexity: ~2x most recent homework problems
  * Building largely from scratch, so much more to do

## Project Timeline

* Weeks 5-6 (this week & next week) - find a partner and brainstorm ideas
    * Come chat in office hours to get early feedback
* Week 7 - propose project & get instructor feedback
* Week 8 - close the loop early and keep developing
* Week 9 - complete initial development & start revising
* Week 10 - finalize/polish project & present

## Project Deliverables

* 2/20 - initial proposal (<1 page)
  * What will generator do and what interface/parameters will it have?
  * Consider how to bootstrap, test, and what features that can be deferred
  * Feedback during 2/21 & 2/23
* 3/4 - Link to working repo (can be feature incomplete)
  * Close the loop early, and build from there
* 3/11 - External (peer) code review
* 3/13 or 3/15 - Presentation
* 3/20 - Links to final repo & revised presentation
  * Following presentation, will have time to make small revisions
  * Encouraged (but not required) to post publicly